# Taller de búsqueda de hiperparámetros

## Hecho por: Leonard David Vivas Dallos


### Parte 1

En este taller vamos a explorar la búsqueda de hiperparámetros de manera automática.

In [1]:
import time
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
X, y = load_breast_cancer(return_X_y=True)

In [3]:
test_size=0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

**Ejemplo 1:** encuentra los mejores hiperparámetros usando `GridSearchCV`

In [4]:
clf = DecisionTreeClassifier()

# Define los valores que usarás en la búsqueda del hiperparametro C
param_grid = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza GridSearchCV
gs = GridSearchCV(clf, param_grid, cv=5, scoring='f1', verbose=1)

t0 = time.time()
gs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Tiempo de búsqueda: 0.865s


In [5]:
np.bincount(y_train) # Nos indicará la cantidad de ejemplos por clase en el conjunto de entrenamiento, como hay un leve desbalanceo, usé `f1` como métrica de evaluación.

array([158, 268])

Podemos ver que el atributo `cv_results_` nos entrega los resultados de toda la búsqueda.

In [6]:
dir(gs)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_run_sea

In [7]:
gs.cv_results_

{'mean_fit_time': array([0.00269046, 0.00568676, 0.00197926, 0.00701842, 0.00306702,
        0.0075995 , 0.00510626, 0.00466471, 0.00438581, 0.00954285,
        0.00446148, 0.00532918, 0.00625577, 0.00567384, 0.00321345,
        0.00265555, 0.00403886, 0.00313439, 0.00883741, 0.00228157,
        0.00883703, 0.00729442, 0.00482497, 0.00513973]),
 'std_fit_time': array([0.00205623, 0.00439541, 0.00166453, 0.00540893, 0.00531628,
        0.0056854 , 0.00647129, 0.0024193 , 0.00225963, 0.00781941,
        0.00617596, 0.0066857 , 0.00766279, 0.0070047 , 0.00642691,
        0.00531111, 0.0039156 , 0.00626879, 0.00728036, 0.00419356,
        0.00736452, 0.00641736, 0.00595238, 0.00629486]),
 'mean_score_time': array([1.12748146e-03, 1.80711746e-03, 6.05106354e-04, 8.30745697e-04,
        2.53920555e-03, 4.05645370e-04, 2.80270576e-03, 3.01990509e-03,
        1.05252266e-03, 4.13942337e-04, 0.00000000e+00, 9.41276550e-04,
        3.12657356e-03, 5.39302826e-05, 3.13267708e-03, 6.57000542e-03,


Lo más importante es extraer los hiperparámetros del modelo que mejor error en de validación sacaron

In [8]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 2}

También es posible el mejor resultado en la métrica usada

In [9]:
gs.best_score_

np.float64(0.9515515581553318)

Finalmente, es posible extraer directamente un estimador que que ha sido creado con los mejores hiperparámetros.

In [10]:
gs.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=2)

**Ejemplo 2:** la clase `RandomizedSearchCV` se puede usar casi de la misma manera, solo que esta vez se debe escoger un número de combinaciones a evaluar; las cuales se escogeran de manera aleatoria.

In [46]:
clf = DecisionTreeClassifier()

# Define los valores que usarás en la búsqueda del hiperparametro C
param_dist = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza RandomizedSearchCV
rs = RandomizedSearchCV(
    clf, 
    param_dist, 
    scoring='f1', 
    cv=5, 
    verbose=1, 
    n_jobs=-1,
    n_iter=10,  # Número de combinaciones aleatorias a probar
    random_state=42
)

t0 = time.time()
rs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Tiempo de búsqueda: 0.082s


In [47]:
rs.best_score_

np.float64(0.9476470698471736)

Podemos ver que aunque se demoró mucho menos, el resultado no es tan bueno.

### Parte 2: Experimento con framework de búsqueda de hiperparámetros extra --> Optuna

In [27]:
# Importar librerías necesarias
import optuna
from sklearn.model_selection import cross_val_score, cross_validate

Como el dataset ya esta cargado, podemos empezar a definir el espacio de búsqueda de hiperparámetros. Para esto, vamos a definir una función objetivo que recibe un objeto `trial` de Optuna. Este objeto se usa para sugerir valores de hiperparámetros. Usaremos el mismo modelo de los anteriores ejemplos, para que sea más fácil comparar los resultados.

In [ ]:
def objective_cv_score(trial):
    # Usar los parámetros predefinidos
    max_depth = trial.suggest_categorical('max_depth', [3, 6, 12, 18])
    min_samples_leaf = trial.suggest_categorical('min_samples_leaf', [1, 2, 3])
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    clf = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion
    )

    # Calcular el score usando validación cruzada
    f1 = cross_val_score(clf, X_train, y_train, scoring='f1', cv=5).mean()
    
    return f1


In [ ]:
# Ejecutar la optimización de hiperparámetros
study_cvs = optuna.create_study(direction='maximize')
study_cvs.optimize(objective_cv_score, n_trials=50)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros usando Optuna con validación cruzada (cross_val_score):")
print(study_cvs.best_params)

# Imprimir el mejor score
print("Mejor score usando Optuna con validación cruzada:")
print(study_cvs.best_value)

[I 2025-07-12 18:54:17,266] A new study created in memory with name: no-name-9cef3a7f-85c9-4a28-bb8f-c7addc140939
[I 2025-07-12 18:54:17,312] Trial 0 finished with value: 0.9316803010006893 and parameters: {'max_depth': 12, 'min_samples_leaf': 3, 'criterion': 'gini'}. Best is trial 0 with value: 0.9316803010006893.
[I 2025-07-12 18:54:17,346] Trial 1 finished with value: 0.9457443208424946 and parameters: {'max_depth': 3, 'min_samples_leaf': 3, 'criterion': 'entropy'}. Best is trial 1 with value: 0.9457443208424946.
[I 2025-07-12 18:54:17,388] Trial 2 finished with value: 0.9430485211318516 and parameters: {'max_depth': 12, 'min_samples_leaf': 3, 'criterion': 'entropy'}. Best is trial 1 with value: 0.9457443208424946.
[I 2025-07-12 18:54:17,425] Trial 3 finished with value: 0.9473315951118393 and parameters: {'max_depth': 18, 'min_samples_leaf': 1, 'criterion': 'entropy'}. Best is trial 3 with value: 0.9473315951118393.
[I 2025-07-12 18:54:17,463] Trial 4 finished with value: 0.9438445

Mejores hiperparámetros usando Optuna con validación cruzada (cross_val_score):
{'max_depth': 3, 'min_samples_leaf': 2, 'criterion': 'entropy'}
Mejor score usando Optuna con validación cruzada:
0.953428640676347


Como pudimos ver en la experimentación con estos mismos, los resultados son bastante parecidos a los que obtuvimos con `GridSearchCV` y `RandomizedSearchCV`. Adicionalmente, Optuna nos permite visualizar el proceso de búsqueda de hiperparámetros, lo que puede ser muy útil para entender cómo se están explorando los espacios de hiperparámetros, y tambien lo hace de manera gráfica.

In [48]:
# Visualizar el estudio
optuna.visualization.plot_optimization_history(study_cvs)

Así, podemos ver como van mejorandos los resultados a medida que se exploran más combinaciones de hiperparámetros, en donde podemos notar que para este caso, para en el intento 25 aproximadamente, se logra el mejor resultado.